# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [156]:
import pandas as pd
import numpy as np
import re
import ipaddress
import math
import unicodedata
import warnings
from scipy import stats
from sklearn.model_selection import train_test_split
from urllib.parse import unquote, urlparse, parse_qs
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Import other libraries if needed

## Import Dataset

In [157]:
# Import PhiUSIIL Phishing URL Dataset
df = pd.read_csv('train.csv') # Import The Dataset Given by The Assistants (From The Spesification Docs)
df_original = df.copy()

# Modify Dataset (Because filename is a feature that's shouldn't be included --> From the metadata of the Dataset)
df = df.drop('FILENAME', axis=1)

# Number of rows and columns
print(f"Dataset shape: {df.shape}")

# Column names and their data types
print("\nColumn info:")
print(df.info())

# Class distribution (legitimate vs phishing)
print("\nClass distribution:")
print(df['label'].value_counts(normalize=True))

# Print The First 5 Rows
df.head()

Dataset shape: (140404, 55)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   URL                         96917 non-null   object 
 2   URLLength                   79765 non-null   float64
 3   Domain                      70207 non-null   object 
 4   DomainLength                94085 non-null   float64
 5   IsDomainIP                  98274 non-null   float64
 6   TLD                         95005 non-null   object 
 7   CharContinuationRate        92362 non-null   float64
 8   TLDLegitimateProb           87531 non-null   float64
 9   URLCharProb                 88333 non-null   float64
 10  TLDLength                   92673 non-null   float64
 11  NoOfSubDomain               96344 non-null   float64
 12  HasObfuscation              74

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,0.036850,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,0.060894,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,0.055829,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [158]:
def correct_data_types(df):
    # Create a dataframe copy to avoid modifying the original
    df_correct = df.copy()

    # Binary features to convert to boolean
    binary_features = [
        'HasTitle', 'HasFavicon', 'HasDescription', 'HasSocialNet',
        'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField',
        'HasCopyrightInfo', 'HasExternalFormSubmit', 'IsDomainIP',
        'IsHTTPS', 'IsResponsive', 'HasObfuscation', 'Bank', 'Pay',
        'Crypto', 'Robots', 'label'
    ]

    # Text features to convert to string
    text_features = ['URL', 'Domain', 'Title']

    # Convert binary features to boolean
    for col in binary_features:
        df_correct[col] = df_correct[col].astype("boolean")
    
    # Convert text features to string
    for col in text_features:
        if col in df_correct.columns:
            df_correct[col] = df_correct[col].astype("string")

    return df_correct

In [159]:
# Apply the function
df_correct = correct_data_types(df)

# Number of rows and columns
print(f"Dataset shape: {df_correct.shape}")

# Column names and their data types
print("\nColumn info:")
print(df_correct.info())

# Class distribution (legitimate vs phishing)
print("\nClass distribution:")
print(df_correct['label'].value_counts(normalize=True))

# Global variables for numerical columns and categorical columns (reusability purposes)
numerical_columns = df_correct.select_dtypes(include=['int64', 'float64']).columns 
categorical_columns = df_correct.select_dtypes(include=['object', 'boolean']).columns
text_columns = df_correct.select_dtypes(include=['string']).columns

# Drop ID from Numerical Columns
numerical_columns = numerical_columns.drop('id')

# Global variables for target columns (The Output Variable)
target_column = 'label'

df_correct.head()

Dataset shape: (140404, 55)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   URL                         96917 non-null   string 
 2   URLLength                   79765 non-null   float64
 3   Domain                      70207 non-null   string 
 4   DomainLength                94085 non-null   float64
 5   IsDomainIP                  98274 non-null   boolean
 6   TLD                         95005 non-null   object 
 7   CharContinuationRate        92362 non-null   float64
 8   TLDLegitimateProb           87531 non-null   float64
 9   URLCharProb                 88333 non-null   float64
 10  TLDLength                   92673 non-null   float64
 11  NoOfSubDomain               96344 non-null   float64
 12  HasObfuscation              74

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,False,NaN,0.800000,NaN,NaN,...,False,False,True,NaN,3.0,NaN,69.0,NaN,NaN,True
1,4,http://uqr.to/1il1z,NaN,<NA>,NaN,<NA>,to,1.000000,0.000896,0.036850,...,<NA>,False,False,NaN,NaN,NaN,NaN,NaN,1.0,False
2,5,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,False,au,0.857143,NaN,0.060894,...,True,False,True,33.0,7.0,8.0,15.0,NaN,2.0,True
3,6,<NA>,31.0,<NA>,NaN,<NA>,com,0.562500,0.522907,0.055829,...,True,False,True,24.0,5.0,14.0,NaN,NaN,NaN,True
4,11,<NA>,NaN,www.nyprowrestling.com,22.0,False,NaN,1.000000,NaN,NaN,...,False,False,True,NaN,NaN,14.0,NaN,0.0,NaN,True


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [160]:
# Split dataset into training and validation set with basic (stratified) splitting method

def create_train_val_split(df, target_col='label', test_size=0.2, random_state=42, verbose=True):
    """
    Creates training and validation splits while preserving class distributions.
    
    Args:
    df: DataFrame containing the complete dataset
    target_col: Name of the target column (default: 'label')
    test_size: Proportion of dataset to include in the validation split (default: 0.2)
    random_state: Random state for reproducibility (default: 42)
    verbose: Whether to print split statistics (default: True)
    
    Returns:
    tuple: (train_data, validation_data)
    """
    
    # Separate features and target
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Create stratified split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )
    
    # Reconstruct complete dataframes with all features including target
    train_data = X_train.copy()
    train_data[target_col] = y_train
    
    validation_data = X_val.copy()
    validation_data[target_col] = y_val
    
    if verbose:
        print("Dataset Split Summary")
        print("-" * 50)
        print(f"\nTotal samples: {len(df)}")
        print(f"Training set size: {len(train_data)} ({len(train_data)/len(df)*100:.2f}%)")
        print(f"Validation set size: {len(validation_data)} ({len(validation_data)/len(df)*100:.2f}%)")
        
        # Class distribution in original dataset
        print("\nOriginal class distribution:")
        orig_dist = df[target_col].value_counts(normalize=True)
        print(f"Legitimate: {orig_dist[True]*100:.2f}%")
        print(f"Phishing: {orig_dist[False]*100:.2f}%")
        
        # Class distribution in training set
        print("\nTraining set class distribution:")
        train_dist = train_data[target_col].value_counts(normalize=True)
        print(f"Legitimate: {train_dist[True]*100:.2f}%")
        print(f"Phishing: {train_dist[False]*100:.2f}%")
        
        # Class distribution in validation set
        print("\nValidation set class distribution:")
        val_dist = validation_data[target_col].value_counts(normalize=True)
        print(f"Legitimate: {val_dist[True]*100:.2f}%")
        print(f"Phishing: {val_dist[False]*100:.2f}%")
        
        # Feature completeness check
        print("\nFeature completeness in training set:")
        train_nulls = train_data.isnull().sum() / len(train_data) * 100
        print(train_nulls.sort_values(ascending=False))
        
        print("\nFeature completeness in validation set:")
        val_nulls = validation_data.isnull().sum() / len(validation_data) * 100
        print(val_nulls.sort_values(ascending=False))
    
    return train_data, validation_data

train_data, validation_data = create_train_val_split(df_correct)

def save_split_datasets(train_data, validation_data, train_path='train_data.csv', val_path='validation_data.csv'):
    """
    Saves the training and validation sets to CSV files
    
    Args:
    train_data: Training DataFrame
    validation_data: Validation DataFrame
    train_path: Path to save training data
    val_path: Path to save validation data
    """
    train_data.to_csv(train_path, index=False)
    validation_data.to_csv(val_path, index=False)
    print(f"\nDatasets saved to {train_path} and {val_path}")

save_split_datasets(train_data, validation_data)

Dataset Split Summary
--------------------------------------------------

Total samples: 140404
Training set size: 112323 (80.00%)
Validation set size: 28081 (20.00%)

Original class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Training set class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Validation set class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Feature completeness in training set:
Domain                        49.964834
NoOfExternalRef               49.454698
LineOfCode                    49.250821
LargestLineLength             48.408607
HasSocialNet                  48.398814
NoOfURLRedirect               48.008867
HasCopyrightInfo              47.967914
NoOfCSS                       47.760476
NoOfObfuscatedChar            47.579748
NoOfSelfRedirect              47.561052
HasPasswordField              47.282391
LetterRatioInURL              46.890664
HasObfuscation                46.739314
ObfuscationRatio              45.832999
NoOfLettersInURL             

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [161]:
# Split dataset into two: dataset with available URL data and dataset without URL data

def split_dataset_by_url(df, verbose=True):
    """
    Splits the dataset into two parts based on URL availability
    
    Args:
    df: Original DataFrame
    verbose: Whether to print summary statistics
    
    Returns:
    tuple: (complete_url_data, missing_url_data)
    """
    # Create the splits
    complete_url_data = df[df['URL'].notna()].copy()
    missing_url_data = df[df['URL'].isna()].copy()
    
    if verbose:
        print("Dataset Split Summary")
        print("-" * 50)
        
        # Overall statistics
        print(f"\nOriginal dataset size: {len(df):,} rows")
        print(f"Complete data size: {len(complete_url_data):,} rows ({len(complete_url_data)/len(df)*100:.2f}%)")
        print(f"Missing URL data size: {len(missing_url_data):,} rows ({len(missing_url_data)/len(df)*100:.2f}%)")
        
        # Class distribution in complete data
        complete_dist = complete_url_data['label'].value_counts(normalize=True)
        print("\nClass distribution in complete data:")
        print(f"Legitimate: {complete_dist[True]*100:.2f}%")
        print(f"Phishing: {complete_dist[False]*100:.2f}%")
        
        # Class distribution in missing URL data
        missing_dist = missing_url_data['label'].value_counts(normalize=True)
        print("\nClass distribution in missing URL data:")
        print(f"Legitimate: {missing_dist[True]*100:.2f}%")
        print(f"Phishing: {missing_dist[False]*100:.2f}%")
        
        # Feature completeness in both datasets
        print("\nFeature completeness in complete data:")
        complete_nulls = complete_url_data.isnull().sum() / len(complete_url_data) * 100
        print(complete_nulls.sort_values(ascending=False))
        
        print("\nFeature completeness in missing URL data:")
        missing_nulls = missing_url_data.isnull().sum() / len(missing_url_data) * 100
        print(missing_nulls.sort_values(ascending=False))
    
    return complete_url_data, missing_url_data

complete_url_data, missing_url_data = split_dataset_by_url(train_data)

def save_split_datasets(complete_url_data, missing_url_data, complete_path='complete_url_data.csv', missing_path='missing_url_data.csv'):
    """
    Saves the split datasets to CSV files
    
    Args:
    complete_url_data: DataFrame with complete URL data
    missing_url_data: DataFrame with missing URL data
    complete_path: Path to save complete data
    missing_path: Path to save missing URL data
    """
    complete_url_data.to_csv(complete_path, index=False)
    missing_url_data.to_csv(missing_path, index=False)
    print(f"\nDatasets saved to {complete_path} and {missing_path}")

save_split_datasets(complete_url_data, missing_url_data)

Dataset Split Summary
--------------------------------------------------

Original dataset size: 112,323 rows
Complete data size: 77,628 rows (69.11%)
Missing URL data size: 34,695 rows (30.89%)

Class distribution in complete data:
Legitimate: 92.53%
Phishing: 7.47%

Class distribution in missing URL data:
Legitimate: 92.38%
Phishing: 7.62%

Feature completeness in complete data:
Domain                        49.953625
NoOfExternalRef               49.466687
LineOfCode                    49.248982
HasSocialNet                  48.389756
LargestLineLength             48.361416
HasCopyrightInfo              47.866749
NoOfURLRedirect               47.844850
NoOfCSS                       47.651621
NoOfObfuscatedChar            47.641315
NoOfSelfRedirect              47.565311
HasPasswordField              47.265162
HasObfuscation                46.850363
LetterRatioInURL              46.792394
ObfuscationRatio              45.685835
NoOfLettersInURL              45.121606
SpacialCharRatio

In [162]:
# Extract features in dataset by deriving the data from available URL data

class URLFeatureExtractor:
    def __init__(self):
        # Regex for IP address detection
        self.ip_pattern = r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$'
        
    def extract_features(self, url):
        """Extract all features from a single URL"""
        if pd.isna(url):
            return {}
            
        try:
            # Parse URL
            parsed = urlparse(url)
            domain = parsed.netloc
            
            # Basic features
            features = {
                'URLLength': len(url),
                'Domain': domain,
                'DomainLength': len(domain),
                'TLD': self._get_tld(domain),
                'TLDLength': len(self._get_tld(domain)),
                'NoOfSubDomain': self._count_subdomains(domain),
                'IsHTTPS': url.startswith('https://'),
                'IsDomainIP': self._is_ip_address(domain)
            }
            
            # Character counts and ratios
            char_features = self._analyze_characters(url)
            features.update(char_features)
            
            # Calculate character continuation rate
            features['CharContinuationRate'] = self._calc_char_continuation(url)
            
            return features
            
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            return {}
    
    def _get_tld(self, domain):
        """Extract TLD from domain"""
        if not domain:
            return ''
        parts = domain.split('.')
        return parts[-1] if len(parts) > 0 else ''
    
    def _count_subdomains(self, domain):
        """Count number of subdomains"""
        if not domain:
            return 0
        # Subtract 1 from dot count (as dots = subdomains + 1)
        return domain.count('.') if domain.count('.') > 0 else 0
    
    def _is_ip_address(self, domain):
        """Check if domain is an IP address"""
        try:
            # Try to create IP address object
            ipaddress.ip_address(domain)
            return True
        except:
            # Check against regex pattern
            return bool(re.match(self.ip_pattern, domain))
    
    def _analyze_characters(self, url):
        """Analyze character types and their ratios in URL"""
        total_len = len(url)
        if total_len == 0:
            return {}
            
        letters = sum(c.isalpha() for c in url)
        digits = sum(c.isdigit() for c in url)
        equals = url.count('=')
        qmarks = url.count('?')
        ampersands = url.count('&')
        
        # Count other special characters
        special_chars = sum(not c.isalnum() for c in url) - equals - qmarks - ampersands
        
        return {
            'NoOfLettersInURL': letters,
            'LetterRatioInURL': letters / total_len,
            'NoOfDegitsInURL': digits,
            'DegitRatioInURL': digits / total_len,
            'NoOfEqualsInURL': equals,
            'NoOfQMarkInURL': qmarks,
            'NoOfAmpersandInURL': ampersands,
            'NoOfOtherSpecialCharsInURL': special_chars,
            'SpacialCharRatioInURL': (special_chars + equals + qmarks + ampersands) / total_len
        }
    
    def _calc_char_continuation(self, url):
        """Calculate character continuation rate"""
        if not url:
            return 0
            
        # Find longest sequences of same type (letter/digit/special)
        def get_type(c):
            if c.isalpha(): return 'alpha'
            if c.isdigit(): return 'digit'
            return 'special'
            
        current_type = get_type(url[0])
        current_length = 1
        max_lengths = {'alpha': 1, 'digit': 1, 'special': 1}
        
        for c in url[1:]:
            c_type = get_type(c)
            if c_type == current_type:
                current_length += 1
                max_lengths[c_type] = max(max_lengths[c_type], current_length)
            else:
                current_type = c_type
                current_length = 1
        
        # Sum of longest sequences divided by URL length
        total_continuation = sum(max_lengths.values())
        return total_continuation / len(url)

In [163]:
# Calculate features in dataset by partially deriving the data from available URL data 

class URLProbabilityFeatures:
    def __init__(self):
        """Initialize probability calculators"""
        self.char_probabilities = {}  # Character frequency dictionary
        self.tld_probabilities = {}   # TLD frequency dictionary
        self.min_probability = 1e-5   # Minimum probability for unknown features
        
    def fit(self, urls, labels):
        """
        Calculate probabilities using legitimate URLs from training data
        
        Args:
        urls: Series of URLs
        labels: Series of labels (True for legitimate, False for phishing)
        """
        # Filter legitimate URLs only
        legitimate_urls = urls[labels == True].dropna()
        
        # Calculate character probabilities
        self._calculate_char_probabilities(legitimate_urls)
        
        # Calculate TLD probabilities
        self._calculate_tld_probabilities(legitimate_urls)
        
    def _calculate_char_probabilities(self, legitimate_urls):
        """Calculate character frequencies from legitimate URLs"""
        # Combine all characters from legitimate URLs
        all_chars = ''.join(legitimate_urls.astype(str).str.lower())
        total_chars = len(all_chars)
        
        # Count each character
        char_counts = {}
        for char in all_chars:
            if char.isalnum():  # Only consider alphanumeric characters
                char_counts[char] = char_counts.get(char, 0) + 1
                
        # Convert to probabilities
        self.char_probabilities = {
            char: count/total_chars 
            for char, count in char_counts.items()
        }
        
        # Print probabilities for verification
        print("Character Probabilities:")
        sorted_probs = dict(sorted(self.char_probabilities.items(), 
                                 key=lambda x: x[1], reverse=True))
        for char, prob in sorted_probs.items():
            print(f"'{char}': {prob:.4f}")
            
    def _calculate_tld_probabilities(self, legitimate_urls):
        """Calculate TLD frequencies from legitimate URLs"""
        # Extract and count TLDs
        tld_counts = {}
        total_tlds = 0
        
        for url in legitimate_urls:
            try:
                tld = urlparse(url).netloc.split('.')[-1].lower()
                tld_counts[tld] = tld_counts.get(tld, 0) + 1
                total_tlds += 1
            except:
                continue
                
        # Convert to probabilities
        self.tld_probabilities = {
            tld: count/total_tlds 
            for tld, count in tld_counts.items()
        }
        
        # Print probabilities for verification
        print("\nTLD Probabilities:")
        sorted_probs = dict(sorted(self.tld_probabilities.items(), 
                                 key=lambda x: x[1], reverse=True))
        for tld, prob in sorted_probs.items():
            print(f"'.{tld}': {prob:.4f}")
            
    def calculate_url_char_prob(self, url):
        """
        Calculate geometric mean of character probabilities
        
        Args:
        url: URL string
        
        Returns:
        float: URL character probability score
        """
        if pd.isna(url):
            return np.nan
            
        try:
            # Get probabilities for each character
            char_probs = []
            for char in str(url).lower():
                if char.isalnum():
                    prob = self.char_probabilities.get(char, self.min_probability)
                    char_probs.append(prob)
                    
            if not char_probs:
                return 0.0
                
            # Calculate geometric mean
            n = len(char_probs)
            log_sum = sum(math.log(p) for p in char_probs)
            return math.exp(log_sum / n)
            
        except:
            return np.nan
            
    def calculate_tld_prob(self, url):
        """
        Get TLD probability or minimum value for unknown TLDs
        
        Args:
        url: URL string
        
        Returns:
        float: TLD legitimacy probability score
        """
        if pd.isna(url):
            return np.nan
            
        try:
            # Extract TLD
            tld = urlparse(url).netloc.split('.')[-1].lower()
            
            # Return probability or minimum value
            return self.tld_probabilities.get(tld, self.min_probability)
            
        except:
            return np.nan
            
    def transform(self, urls):
        """
        Calculate both probability features for a series of URLs
        
        Args:
        urls: Series of URLs
        
        Returns:
        DataFrame with URLCharProb and TLDLegitimateProb columns
        """
        return pd.DataFrame({
            'URLCharProb': urls.apply(self.calculate_url_char_prob),
            'TLDLegitimateProb': urls.apply(self.calculate_tld_prob)
        })

class URLObfuscationDetector:
    def __init__(self):
        """Initialize detector with patterns"""
        # URL encoding patterns
        self.url_encoding_pattern = r'%[0-9A-Fa-f]{2}'
        self.double_encoding_pattern = r'%25[0-9A-Fa-f]{2}'
        
        # Unicode/punycode patterns
        self.punycode_pattern = r'xn--'
        
        # Common character substitutions
        self.substitutions = {
            '0': 'o',
            '1': 'l',
            '3': 'e',
            '4': 'a',
            '5': 's',
            '6': 'b',
            '7': 't',
            '8': 'b',
            '9': 'g'
        }
        
        # Similar-looking Unicode characters
        self.unicode_substitutions = {
            'а': 'a',  # Cyrillic 'a'
            'е': 'e',  # Cyrillic 'e'
            'о': 'o',  # Cyrillic 'o'
            'р': 'p',  # Cyrillic 'p'
            'с': 'c',  # Cyrillic 'c'
            'ѕ': 's',  # Cyrillic 's'
            'і': 'i',  # Cyrillic 'i'
        }
        
    def detect_obfuscation(self, url):
        """
        Detect various types of URL obfuscation
        
        Args:
        url: URL string
        
        Returns:
        dict: Detection results and counts
        """
        if pd.isna(url):
            return {
                'HasObfuscation': False,
                'NoOfObfuscatedChar': 0,
                'ObfuscationRatio': 0.0,
                'techniques': []
            }
            
        url = str(url)
        techniques = []
        obfuscated_chars = 0
        
        # Check URL encoding
        url_encoded_chars = self._detect_url_encoding(url)
        if url_encoded_chars > 0:
            techniques.append('url_encoding')
            obfuscated_chars += url_encoded_chars
            
        # Check double encoding
        double_encoded_chars = self._detect_double_encoding(url)
        if double_encoded_chars > 0:
            techniques.append('double_encoding')
            obfuscated_chars += double_encoded_chars
            
        # Check punycode
        if self._detect_punycode(url):
            techniques.append('punycode')
            # Count entire domain as obfuscated if punycode is used
            domain = urlparse(url).netloc
            obfuscated_chars += len(domain)
            
        # Check character substitutions
        substituted_chars = self._detect_char_substitution(url)
        if substituted_chars > 0:
            techniques.append('char_substitution')
            obfuscated_chars += substituted_chars
            
        # Check Unicode substitutions
        unicode_chars = self._detect_unicode_substitution(url)
        if unicode_chars > 0:
            techniques.append('unicode_substitution')
            obfuscated_chars += unicode_chars
            
        # Calculate ratio
        total_chars = len(url)
        obfuscation_ratio = obfuscated_chars / total_chars if total_chars > 0 else 0
        
        return {
            'HasObfuscation': len(techniques) > 0,
            'NoOfObfuscatedChar': obfuscated_chars,
            'ObfuscationRatio': obfuscation_ratio,
            'techniques': techniques
        }
        
    def _detect_url_encoding(self, url):
        """Detect standard URL encoding"""
        matches = re.findall(self.url_encoding_pattern, url)
        return len(matches)
        
    def _detect_double_encoding(self, url):
        """Detect double URL encoding"""
        matches = re.findall(self.double_encoding_pattern, url)
        return len(matches)
        
    def _detect_punycode(self, url):
        """Detect punycode encoding in domain"""
        return self.punycode_pattern in url.lower()
        
    def _detect_char_substitution(self, url):
        """Detect numeric/character substitutions"""
        count = 0
        domain = urlparse(url).netloc.lower()
        
        for digit, letter in self.substitutions.items():
            # Check if digit appears in domain portion
            digit_count = domain.count(digit)
            if digit_count > 0:
                # Verify it's likely a substitution by checking surrounding chars
                for idx in range(len(domain) - 1):
                    if domain[idx] == digit:
                        # Check if surrounded by letters
                        if (idx > 0 and domain[idx-1].isalpha()) or \
                           (idx < len(domain)-1 and domain[idx+1].isalpha()):
                            count += 1
                            
        return count
        
    def _detect_unicode_substitution(self, url):
        """Detect Unicode character substitutions"""
        count = 0
        for char in url:
            # Check if character has a similar-looking ASCII version
            if char in self.unicode_substitutions:
                count += 1
        return count

In [164]:
# Fill dataset with URL-derived features

def fill_url_derived_features(df_original, train_mode=True):
    """
    Fill URL-based and URL-partially based features
    
    Args:
    df_original: Original DataFrame
    train_mode: If True, use this data to train probability calculator
    
    Returns:
    DataFrame with filled features
    """
    # Create copy to avoid modifying original
    df = df_original.copy()
    
    # Initialize extractor
    extractor = URLFeatureExtractor()
    
    # Get URLs that need processing (have at least one missing derived feature)
    derived_features = [
        'URLLength', 'Domain', 'DomainLength', 'TLD', 'TLDLength',
        'NoOfSubDomain', 'IsHTTPS', 'IsDomainIP', 'NoOfLettersInURL',
        'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',
        'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',
        'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL',
        'CharContinuationRate'
    ]
    
    # Process each URL with missing features
    for idx, row in df.iterrows():
        if row['URL'] is not None and any(pd.isna(row[feat]) for feat in derived_features):
            features = extractor.extract_features(row['URL'])
            for feat, value in features.items():
                if pd.isna(df.at[idx, feat]):
                    df.at[idx, feat] = value

    # Initialize feature calculators
    prob_calculator = URLProbabilityFeatures()
    obfuscation_detector = URLObfuscationDetector()
    
    # Train probability calculator if in train mode
    if train_mode:
        prob_calculator.fit(df['URL'], df['label'])
    
    # Process URLs in batches to avoid memory issues
    batch_size = 1000
    total_rows = len(df)
    
    for start_idx in range(0, total_rows, batch_size):
        end_idx = min(start_idx + batch_size, total_rows)
        batch_df = df.iloc[start_idx:end_idx]
        
        # Calculate probability features
        if 'URLCharProb' in df.columns or 'TLDLegitimateProb' in df.columns:
            prob_features = prob_calculator.transform(batch_df['URL'])
            
            # Fill missing values in URLCharProb
            if 'URLCharProb' in df.columns:
                mask = df.iloc[start_idx:end_idx]['URLCharProb'].isna()
                df.loc[df.index[start_idx:end_idx][mask], 'URLCharProb'] = \
                    prob_features.loc[mask, 'URLCharProb']
            
            # Fill missing values in TLDLegitimateProb
            if 'TLDLegitimateProb' in df.columns:
                mask = df.iloc[start_idx:end_idx]['TLDLegitimateProb'].isna()
                df.loc[df.index[start_idx:end_idx][mask], 'TLDLegitimateProb'] = \
                    prob_features.loc[mask, 'TLDLegitimateProb']
        
        # Detect obfuscation
        for idx, row in batch_df.iterrows():
            if pd.notna(row['URL']):
                obfuscation_results = obfuscation_detector.detect_obfuscation(row['URL'])
                
                # Fill HasObfuscation
                if 'HasObfuscation' in df.columns and pd.isna(df.at[idx, 'HasObfuscation']):
                    df.at[idx, 'HasObfuscation'] = obfuscation_results['HasObfuscation']
                
                # Fill NoOfObfuscatedChar
                if 'NoOfObfuscatedChar' in df.columns and pd.isna(df.at[idx, 'NoOfObfuscatedChar']):
                    df.at[idx, 'NoOfObfuscatedChar'] = obfuscation_results['NoOfObfuscatedChar']
                
                # Fill ObfuscationRatio
                if 'ObfuscationRatio' in df.columns and pd.isna(df.at[idx, 'ObfuscationRatio']):
                    df.at[idx, 'ObfuscationRatio'] = obfuscation_results['ObfuscationRatio']
    
    return df

derived_url_data = fill_url_derived_features(complete_url_data)

def analyze_filling_results(df_original, df_filled):
    """
    Analyze the results of feature filling
    
    Args:
    df_original: Original DataFrame
    df_filled: Filled DataFrame
    """
    print("Feature Filling Analysis")
    print("-" * 50)

    # Features to analyze
    derived_features = [
        'URLLength', 'Domain', 'DomainLength', 'TLD', 'TLDLength',
        'NoOfSubDomain', 'IsHTTPS', 'IsDomainIP', 'NoOfLettersInURL',
        'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',
        'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',
        'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL',
        'CharContinuationRate'
    ]
    prob_features = ['URLCharProb', 'TLDLegitimateProb']
    obfuscation_features = ['HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio']

    # Analyze URL-derived features
    print("\nURL-derived Features:")
    for feature in derived_features:
        if feature in df_original.columns:
            orig_missing = df_original[feature].isna().sum()
            filled_missing = df_filled[feature].isna().sum()
            filled_count = orig_missing - filled_missing

            print(f"\n{feature}:")
            print(f"Original missing values: {orig_missing}")
            print(f"Remaining missing values: {filled_missing}")
            print(f"Filled values: {filled_count}")

            if filled_count > 0:
                print("\nDistribution of filled values:")
                filled_mask = df_original[feature].isna() & df_filled[feature].notna()
                print(df_filled.loc[filled_mask, feature].describe())

    # Analyze probability features
    print("\nProbability Features:")
    for feature in prob_features:
        if feature in df_original.columns:
            orig_missing = df_original[feature].isna().sum()
            filled_missing = df_filled[feature].isna().sum()
            filled_count = orig_missing - filled_missing
            
            print(f"\n{feature}:")
            print(f"Original missing: {orig_missing}")
            print(f"Remaining missing: {filled_missing}")
            print(f"Filled values: {filled_count}")
            
            if filled_count > 0:
                print("\nDistribution of filled values:")
                filled_mask = df_original[feature].isna() & df_filled[feature].notna()
                print(df_filled.loc[filled_mask, feature].describe())
    
    # Analyze obfuscation features
    print("\nObfuscation Features:")
    for feature in obfuscation_features:
        if feature in df_original.columns:
            orig_missing = df_original[feature].isna().sum()
            filled_missing = df_filled[feature].isna().sum()
            filled_count = orig_missing - filled_missing
            
            print(f"\n{feature}:")
            print(f"Original missing: {orig_missing}")
            print(f"Remaining missing: {filled_missing}")
            print(f"Filled values: {filled_count}")
            
            if filled_count > 0:
                if feature == 'HasObfuscation':
                    filled_mask = df_original[feature].isna() & df_filled[feature].notna()
                    value_counts = df_filled.loc[filled_mask, feature].value_counts()
                    print("\nDistribution of filled values:")
                    print(value_counts)
                else:
                    print("\nDistribution of filled values:")
                    filled_mask = df_original[feature].isna() & df_filled[feature].notna()
                    print(df_filled.loc[filled_mask, feature].describe())

analyze_filling_results(complete_url_data, derived_url_data)

def save_url_derived_datasets(derived_data, derived_path='derived_url_data.csv'):
    """
    Saves the URL-derived dataset to CSV files
    
    Args:
    complete_data: DataFrame with complete URL data
    missing_url_data: DataFrame with missing URL data
    complete_path: Path to save complete data
    missing_path: Path to save missing URL data
    """
    derived_data.to_csv(derived_path, index=False)
    print(f"\nDatasets saved to {derived_path}")

save_url_derived_datasets(derived_url_data)

Character Probabilities:
'w': 0.1157
't': 0.1016
's': 0.0649
'o': 0.0568
'h': 0.0486
'p': 0.0483
'e': 0.0448
'c': 0.0404
'a': 0.0402
'r': 0.0349
'm': 0.0333
'i': 0.0321
'n': 0.0291
'l': 0.0219
'u': 0.0167
'g': 0.0161
'd': 0.0149
'b': 0.0092
'k': 0.0082
'f': 0.0073
'y': 0.0073
'v': 0.0061
'j': 0.0026
'z': 0.0023
'x': 0.0016
'q': 0.0007
'2': 0.0004
'1': 0.0004
'0': 0.0003
'4': 0.0002
'3': 0.0002
'5': 0.0001
'6': 0.0001
'9': 0.0001
'7': 0.0001
'8': 0.0001

TLD Probabilities:
'.com': 0.5102
'.org': 0.1240
'.uk': 0.0451
'.net': 0.0290
'.de': 0.0243
'.au': 0.0192
'.jp': 0.0148
'.edu': 0.0141
'.it': 0.0124
'.ca': 0.0112
'.nl': 0.0112
'.fr': 0.0099
'.gov': 0.0094
'.br': 0.0087
'.ru': 0.0066
'.in': 0.0061
'.us': 0.0052
'.eu': 0.0051
'.info': 0.0050
'.be': 0.0045
'.se': 0.0042
'.cz': 0.0041
'.es': 0.0041
'.pl': 0.0040
'.nz': 0.0040
'.co': 0.0037
'.ch': 0.0036
'.ie': 0.0032
'.at': 0.0032
'.gr': 0.0032
'.io': 0.0031
'.no': 0.0030
'.ro': 0.0029
'.za': 0.0025
'.fi': 0.0024
'.mil': 0.0023
'.tr': 0.00

In [165]:
import pandas as pd
import numpy as np
import warnings
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import chi2_contingency
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, Any

@dataclass
class FeatureStats:
    """Data class to store feature statistics"""
    data_type: str
    distribution_type: Optional[str] = None
    imputation_strategy: str = ""
    reason: str = ""
    top_correlations: Optional[Dict[str, float]] = None

@dataclass
class BooleanImputationStats:
    """Data class untuk menyimpan statistik imputasi boolean"""
    original_count: int
    missing_count: int
    true_ratio: float
    imputation_strategy: str
    imputed_value: bool

class DataPreprocessor:
    """Class for comprehensive data analysis and imputation"""

    def __init__(self, df: pd.DataFrame, verbose: int = 1):
        self.df = df.copy()
        self.verbose = verbose
        self.numerical_columns, self.categorical_columns = self._define_column_types()
        self.strategy_summary = {}
        self.boolean_columns = self._get_boolean_columns()
        self.boolean_stats = {}
        warnings.filterwarnings('ignore', category=FutureWarning)

    def _get_boolean_columns(self) -> list:
        """Identify boolean columns in DataFrame"""
        return list(self.df.select_dtypes(include=['bool', 'boolean']).columns)

    def _analyze_boolean_distribution(self, column: str) -> Tuple[float, str, bool]:
        """Analyze boolean distribution and determine imputation strategy"""
        true_ratio = self.df[column].mean(skipna=True)

        if pd.isna(true_ratio):
            strategy = "default_false"
            imputed_value = False
        elif true_ratio == 0.5:
            strategy = "balanced_false"
            imputed_value = False
        elif true_ratio > 0.75:
            strategy = "majority_true"
            imputed_value = True
        elif true_ratio < 0.25:
            strategy = "majority_false"
            imputed_value = False
        else:
            strategy = "mode"
            imputed_value = true_ratio > 0.5

        return true_ratio, strategy, imputed_value
    
    def _define_column_types(self) -> Tuple[pd.Index, pd.Index]:
        """Identify numerical and categorical columns"""
        numerical = self.df.select_dtypes(include=['int64', 'float64']).columns
        categorical = self.df.select_dtypes(include=['object', 'boolean', 'bool']).columns

        # Exclude non-analytical columns
        numerical = numerical.drop('id', errors='ignore')
        categorical = categorical.drop(['URL', 'Domain', 'FILENAME', 'Title'], errors='ignore')

        return numerical, categorical
    
    def analyze_missing_values(self) -> pd.DataFrame:
        """Analyze missing values in the dataset"""
        self._print_section("Missing Values Analysis")

        missing_stats = pd.DataFrame({
            'Data Type': self.df.dtypes,
            'Missing Count': self.df.isnull().sum(),
            'Missing Percentage': (self.df.isnull().sum() / len(self.df) * 100).round(2)
        }).sort_values('Missing Percentage', ascending=False)

        if self.verbose > 0:
            print("\nMissing Values Summary:")
            with pd.option_context('display.max_rows', None):
                print(missing_stats.to_string())

        return missing_stats

    def _print_section(self, title: str) -> None:
        """Print formatted section header"""
        if self.verbose > 0:
            print(f"\n{'='*50}\n{title.center(50)}\n{'='*50}")



    def impute_boolean_values(self, df_imputed: pd.DataFrame) -> pd.DataFrame:
        """Handle boolean imputation specifically"""
        if self.verbose > 0:
            print("\nMemulai imputasi kolom boolean...")

        for column in self.boolean_columns:
            if self.verbose > 1:
                print(f"\nMemproses kolom boolean: {column}")

            missing_count = df_imputed[column].isnull().sum()

            if missing_count == 0:
                continue

            try:
                # Analyze distribution and determine strategy
                true_ratio, strategy, imputed_value = self._analyze_boolean_distribution(column)

                # Perform imputation
                df_imputed[column] = df_imputed[column].fillna(imputed_value)

                # Ensure boolean type
                df_imputed[column] = df_imputed[column].astype('boolean')

                # Store statistics
                self.boolean_stats[column] = BooleanImputationStats(
                    original_count=len(df_imputed[column]),
                    missing_count=missing_count,
                    true_ratio=true_ratio,
                    imputation_strategy=strategy,
                    imputed_value=imputed_value
                )

                if self.verbose > 0:
                    self._print_boolean_summary(column)

            except Exception as e:
                if self.verbose > 0:
                    print(f"Error pada imputasi kolom boolean {column}: {str(e)}")
                    print("Menggunakan nilai default False")
                df_imputed[column] = df_imputed[column].fillna(False).astype('boolean')

        return df_imputed
    
    def analyze_feature_distributions(self) -> Dict[str, FeatureStats]:
        """Analyze distributions and determine imputation strategies"""
        self._print_section("Distribution Analysis")

        # Analyze numerical features
        for col in self.numerical_columns:
            clean_data = self.df[col].dropna()
            if len(clean_data) > 0:
                stats = {
                    'mean': clean_data.mean(),
                    'median': clean_data.median(),
                    'std': clean_data.std(),
                    'skew': clean_data.skew(),
                    'kurtosis': clean_data.kurtosis()
                }

                distribution_type = self._interpret_distribution(stats)

                correlations = self.df[self.numerical_columns].corr()[col].abs().sort_values(ascending=False)
                # Adjust correlation thresholds to 0.75 (strong correlation as per rules of thumb)
                top_corr = correlations[correlations >= 0.75].drop(col, errors='ignore')

                strategy = "Imputation using Predictive Models" if len(top_corr) > 0 else stats['suggested_strategy']
                reason = (f"Strong correlations with {', '.join(top_corr.index)}" 
                        if len(top_corr) > 0 
                        else f"Based on {distribution_type}")

                self.strategy_summary[col] = FeatureStats(
                    data_type=str(self.df[col].dtype),
                    distribution_type=distribution_type,
                    imputation_strategy=strategy,
                    reason=reason,
                    top_correlations=top_corr[:5].to_dict()
                )

        # Analyze categorical features
        for col in self.categorical_columns:
            if self.df[col].isnull().any():
                self.strategy_summary[col] = FeatureStats(
                    data_type=str(self.df[col].dtype),
                    imputation_strategy="Mode Imputation",
                    reason="Standard approach for categorical data"
                )

        if self.verbose > 0:
            self._print_summary()

        return self.strategy_summary


    def _print_boolean_summary(self, column: str) -> None:
        """Print boolean imputation summary"""
        stats = self.boolean_stats[column]
        print(f"\nRingkasan imputasi boolean untuk {column}:")
        print(f"  - Jumlah nilai hilang: {stats.missing_count}")
        print(f"  - Rasio True: {stats.true_ratio:.2%}")
        print(f"  - Strategi: {stats.imputation_strategy}")
        print(f"  - Nilai imputasi: {stats.imputed_value}")

    def impute_missing_values(self) -> pd.DataFrame:
        """
        Handle missing values for both boolean and non-boolean columns
        and combine the results into a single DataFrame.
        """
        warnings.filterwarnings('ignore', category=FutureWarning)
        df_imputed = self.df.copy().replace({pd.NA: np.nan})

        if self.verbose > 0:
            print("\n" + "=" * 50)
            print("ADVANCED IMPUTATION STRATEGIES".center(50))
            print("=" * 50)

        # Handle boolean columns
        if self.boolean_columns:
            df_imputed = self.impute_boolean_values(df_imputed)

        # Handle non-boolean columns
        imputed_columns = []

        for col, details in self.strategy_summary.items():
            if col in self.boolean_columns:
                continue  # Skip boolean columns as they're already handled

            if self.verbose > 0:
                print(f"\nHandling column: {col} (Data Type: {details.data_type})")

            missing_count = df_imputed[col].isnull().sum()
            if missing_count == 0:
                continue  # Skip columns without missing values

            try:
                # Apply the suggested imputation strategy
                if details.imputation_strategy == "Mean Imputation":
                    imputer = SimpleImputer(strategy="mean")
                elif details.imputation_strategy == "Median Imputation":
                    imputer = SimpleImputer(strategy="median")
                elif details.imputation_strategy == "Mode Imputation":
                    imputer = SimpleImputer(strategy="most_frequent")
                else:
                    # For more complex strategies like predictive modeling
                    imputer = IterativeImputer(max_iter=10, random_state=0)

                # Fit and transform the column
                df_imputed[col] = imputer.fit_transform(df_imputed[[col]])

                imputed_columns.append(col)
            except Exception as e:
                print(f"Error imputing column {col}: {str(e)}")

        if self.verbose > 0:
            print("\nImputation process completed successfully.")
            print("Successfully imputed columns:")
            print(imputed_columns)

        return df_imputed


    def get_boolean_imputation_report(self) -> pd.DataFrame:
        """Generate boolean imputation report"""
        report_data = []
        for column, stats in self.boolean_stats.items():
            report_data.append({
                'Column': column,
                'Missing Values': stats.missing_count,
                'True Ratio': f"{stats.true_ratio:.2%}",
                'Strategy': stats.imputation_strategy,
                'Imputed Value': stats.imputed_value
            })

        return pd.DataFrame(report_data)

    def _interpret_distribution(self, stats_dict: Dict[str, float]) -> str:
        """Interpret statistical distributions"""
        interpretation = []

        # Perbedaan antara mean dan median terhadap standar deviasi
        mean_median_diff = abs(stats_dict['mean'] - stats_dict['median']) / stats_dict['std']
        if mean_median_diff < 0.2:
            interpretation.append("Relatively Symmetric (Normal Distribution)")
            stats_dict['suggested_strategy'] = "Mean Imputation"
        else:
            direction = "Right" if stats_dict['mean'] > stats_dict['median'] else "Left"
            interpretation.append(f"Skewed to {direction}")
            stats_dict['suggested_strategy'] = "Median Imputation"

        # Analisis skewness
        if abs(stats_dict['skew']) < 0.5:
            interpretation.append("Near-Symmetric Distribution")
        elif abs(stats_dict['skew']) < 1:
            interpretation.append("Moderate Skewness")
        else:
            interpretation.append("High Skewness")

        return ", ".join(interpretation)
    
    def _print_summary(self) -> None:
        """Print formatted analysis summary"""
        print("\nFeature Imputation Summary:")
        for col, stats in self.strategy_summary.items():
            print(f"\nColumn: {col}")
            print(f"    Data Type: {stats.data_type}")
            if stats.distribution_type:
                print(f"    Distribution: {stats.distribution_type}")
            print(f"    Imputation Strategy: {stats.imputation_strategy}")
            print(f"    Reason: {stats.reason}")
            if stats.top_correlations:
                print("    Top Correlations:")
                for feature, corr in stats.top_correlations.items():
                    print(f"        - {feature}: {corr:.3f}")

    def _validate_imputation(self, df_before: pd.DataFrame, df_after: pd.DataFrame) -> None:
        """
        Validate the imputation process by comparing the number of missing values
        before and after imputation.

        Args:
            df_before: DataFrame before imputation.
            df_after: DataFrame after imputation.
        """
        missing_before = df_before.isnull().sum()
        missing_after = df_after.isnull().sum()

        comparison = pd.DataFrame({
            'Missing Before': missing_before,
            'Missing After': missing_after,
            'Values Imputed': missing_before - missing_after
        })

        print("\nImputation Validation:")
        print(comparison[comparison['Values Imputed'] > 0])




def process_dataset(df: pd.DataFrame, verbose: int = 1) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Main function to analyze and process dataset

    Args:
        df: Input DataFrame
        verbose: Verbosity level

    Returns:
        Tuple containing processed DataFrame and analysis results
    """
    processor = DataPreprocessor(df, verbose)

    # Analyze missing values and distributions
    missing_analysis = processor.analyze_missing_values()
    feature_analysis = processor.analyze_feature_distributions()

    # Perform imputation
    df_processed = processor.impute_missing_values()

    results = {
        'missing_analysis': missing_analysis,
        'feature_analysis': feature_analysis,
        'boolean_imputation_report': processor.get_boolean_imputation_report()
    }

    return df_processed, results



# Process the dataset (analysis + imputation)
df_completed, results = process_dataset(derived_url_data, verbose=1)

# Access specific results
missing_analysis = results['missing_analysis']
feature_analysis = results['feature_analysis']
boolean_report = results['boolean_imputation_report']

# Save results
def save_completed_datasets(completed_data, completed_path='completed_data.csv'):
    """
    Saves the completed dataset to CSV files

    Args:
        completed_data: DataFrame with complete URL data
        completed_path: Path to save complete data
    """
    completed_data.to_csv(completed_path, index=False)
    print(f"\nDatasets saved to {completed_path}")

save_completed_datasets(df_completed)



             Missing Values Analysis              

Missing Values Summary:
                                 Data Type  Missing Count  Missing Percentage
NoOfExternalRef                    float64          38400               49.47
LineOfCode                         float64          38231               49.25
HasSocialNet                       boolean          37564               48.39
LargestLineLength                  float64          37542               48.36
HasCopyrightInfo                   boolean          37158               47.87
NoOfURLRedirect                    float64          37141               47.84
NoOfCSS                            float64          36991               47.65
NoOfSelfRedirect                   float64          36924               47.57
HasPasswordField                   boolean          36691               47.27
HasSubmitButton                    boolean          34078               43.90
NoOfJS                             float64          33492        

In [166]:
import pandas as pd

# Muat dataset dari file CSV
derived_url_data = pd.read_csv("derived_url_data.csv")

# Periksa apakah ada nilai yang hilang
print(derived_url_data.isnull().sum())


id                                0
URL                               0
URLLength                         0
Domain                            0
DomainLength                      0
IsDomainIP                        0
TLD                               0
CharContinuationRate              0
TLDLegitimateProb                 0
URLCharProb                       0
TLDLength                         0
NoOfSubDomain                     0
HasObfuscation                    0
NoOfObfuscatedChar                0
ObfuscationRatio                  0
NoOfLettersInURL                  0
LetterRatioInURL                  0
NoOfDegitsInURL                   0
DegitRatioInURL                   0
NoOfEqualsInURL                   0
NoOfQMarkInURL                    0
NoOfAmpersandInURL                0
NoOfOtherSpecialCharsInURL        0
SpacialCharRatioInURL             0
IsHTTPS                           0
LineOfCode                    38231
LargestLineLength             37542
HasTitle                    

In [167]:
def ensure_boolean_columns(df: pd.DataFrame, boolean_columns: list) -> pd.DataFrame:
    """Konversi kolom boolean agar sesuai tipe data"""
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype("boolean")
    return df

In [168]:
boolean_columns = [
    "IsDomainIP", "HasObfuscation", "IsHTTPS", "HasTitle", "HasFavicon", "Robots", 
    "IsResponsive", "HasDescription", "HasExternalFormSubmit", "HasSocialNet",
    "HasSubmitButton", "HasHiddenFields", "HasPasswordField", "Bank", "Pay", 
    "Crypto", "HasCopyrightInfo", "label"
]
derived_url_data = ensure_boolean_columns(derived_url_data, boolean_columns)

In [169]:
processor = DataPreprocessor(derived_url_data, verbose=1)

In [170]:
missing_analysis = processor.analyze_missing_values()


             Missing Values Analysis              

Missing Values Summary:
                           Data Type  Missing Count  Missing Percentage
NoOfExternalRef              float64          38400               49.47
LineOfCode                   float64          38231               49.25
HasSocialNet                 boolean          37564               48.39
LargestLineLength            float64          37542               48.36
HasCopyrightInfo             boolean          37158               47.87
NoOfURLRedirect              float64          37141               47.84
NoOfCSS                      float64          36991               47.65
NoOfSelfRedirect             float64          36924               47.57
HasPasswordField             boolean          36691               47.27
HasSubmitButton              boolean          34078               43.90
NoOfJS                       float64          33492               43.14
HasFavicon                   boolean          32397        

In [171]:
feature_analysis = processor.analyze_feature_distributions()


              Distribution Analysis               

Feature Imputation Summary:

Column: URLLength
    Data Type: float64
    Distribution: Relatively Symmetric (Normal Distribution), High Skewness
    Imputation Strategy: Imputation using Predictive Models
    Reason: Strong correlations with NoOfLettersInURL, NoOfDegitsInURL
    Top Correlations:
        - NoOfLettersInURL: 0.943
        - NoOfDegitsInURL: 0.811

Column: DomainLength
    Data Type: float64
    Distribution: Relatively Symmetric (Normal Distribution), High Skewness
    Imputation Strategy: Mean Imputation
    Reason: Based on Relatively Symmetric (Normal Distribution), High Skewness

Column: CharContinuationRate
    Data Type: float64
    Distribution: Skewed to Left, Near-Symmetric Distribution
    Imputation Strategy: Median Imputation
    Reason: Based on Skewed to Left, Near-Symmetric Distribution

Column: TLDLegitimateProb
    Data Type: float64
    Distribution: Skewed to Left, Near-Symmetric Distribution
    I

In [172]:
df_completed = processor.impute_missing_values()


          ADVANCED IMPUTATION STRATEGIES          

Memulai imputasi kolom boolean...

Ringkasan imputasi boolean untuk HasTitle:
  - Jumlah nilai hilang: 24673
  - Rasio True: 97.49%
  - Strategi: majority_true
  - Nilai imputasi: True

Ringkasan imputasi boolean untuk HasFavicon:
  - Jumlah nilai hilang: 32397
  - Rasio True: 53.35%
  - Strategi: mode
  - Nilai imputasi: True

Ringkasan imputasi boolean untuk Robots:
  - Jumlah nilai hilang: 26135
  - Rasio True: 38.90%
  - Strategi: mode
  - Nilai imputasi: False

Ringkasan imputasi boolean untuk IsResponsive:
  - Jumlah nilai hilang: 23489
  - Rasio True: 81.15%
  - Strategi: majority_true
  - Nilai imputasi: True

Ringkasan imputasi boolean untuk HasDescription:
  - Jumlah nilai hilang: 30150
  - Rasio True: 68.60%
  - Strategi: mode
  - Nilai imputasi: True

Ringkasan imputasi boolean untuk HasExternalFormSubmit:
  - Jumlah nilai hilang: 30771
  - Rasio True: 6.83%
  - Strategi: majority_false
  - Nilai imputasi: False

Ringkasa

In [173]:
processor._validate_imputation(derived_url_data, df_completed)


Imputation Validation:
                       Missing Before  Missing After  Values Imputed
LineOfCode                      38231              0           38231
LargestLineLength               37542              0           37542
HasTitle                        24673              0           24673
DomainTitleMatchScore           27558              0           27558
URLTitleMatchScore              28859              0           28859
HasFavicon                      32397              0           32397
Robots                          26135              0           26135
IsResponsive                    23489              0           23489
NoOfURLRedirect                 37141              0           37141
NoOfSelfRedirect                36924              0           36924
HasDescription                  30150              0           30150
NoOfPopup                       23797              0           23797
NoOfiFrame                      27473              0           27473
HasExterna

In [174]:
df_completed.to_csv("completed_data.csv", index=False)
print("\nHasil imputasi lengkap disimpan di 'completed_data.csv'")


Hasil imputasi lengkap disimpan di 'completed_data.csv'


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [175]:
# Write your code here

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [176]:
duplicate_urls = df_completed['URL'].value_counts()
duplicate_urls = duplicate_urls[duplicate_urls > 1].index
print(duplicate_urls)

Index(['http://test-mantenimiento-bancaweb.azurewebsites.net/'], dtype='object', name='URL')


In [177]:
rows_with_duplicate_urls = df_completed[df_completed['URL'].isin(duplicate_urls)]
print(rows_with_duplicate_urls)


           id                                                URL  URLLength  \
16778  122751  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   
21282  180616  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   

                                              Domain  DomainLength  \
16778  test-mantenimiento-bancaweb.azurewebsites.net          45.0   
21282  test-mantenimiento-bancaweb.azurewebsites.net          45.0   

       IsDomainIP  TLD  CharContinuationRate  TLDLegitimateProb  URLCharProb  \
16778       False  net              0.320755           0.029014     0.061098   
21282       False  net              0.341463           0.029014     0.061098   

       ...    Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS     NoOfJS  \
16778  ...  False   False              True  41.483831      1.0  16.370174   
21282  ...  False   False              True   0.000000      5.0   1.000000   

       NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
16778          1

In [178]:
rows_with_duplicate_urls = df[df['URL'].isin(duplicate_urls)]
print(rows_with_duplicate_urls)

            id                                                URL  URLLength  \
73353   122751  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   
107606  180616  http://test-mantenimiento-bancaweb.azurewebsit...        NaN   

                                               Domain  DomainLength  \
73353                                             NaN          45.0   
107606  test-mantenimiento-bancaweb.azurewebsites.net          45.0   

        IsDomainIP  TLD  CharContinuationRate  TLDLegitimateProb  URLCharProb  \
73353          0.0  net                   NaN                NaN     0.061098   
107606         0.0  NaN              0.341463                NaN     0.061098   

        ...  Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  \
73353   ...  NaN     NaN               1.0        NaN      1.0     NaN   
107606  ...  NaN     0.0               1.0        0.0      NaN     1.0   

        NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
73353           1.

In [179]:
rows_with_duplicate_urls = df_completed[df_completed['URL'].isin(duplicate_urls)]
print(rows_with_duplicate_urls)

           id                                                URL  URLLength  \
16778  122751  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   
21282  180616  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   

                                              Domain  DomainLength  \
16778  test-mantenimiento-bancaweb.azurewebsites.net          45.0   
21282  test-mantenimiento-bancaweb.azurewebsites.net          45.0   

       IsDomainIP  TLD  CharContinuationRate  TLDLegitimateProb  URLCharProb  \
16778       False  net              0.320755           0.029014     0.061098   
21282       False  net              0.341463           0.029014     0.061098   

       ...    Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS     NoOfJS  \
16778  ...  False   False              True  41.483831      1.0  16.370174   
21282  ...  False   False              True   0.000000      5.0   1.000000   

       NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
16778          1

In [180]:
def merge_duplicate_urls(df_completed: pd.DataFrame, df_original: pd.DataFrame) -> pd.DataFrame:
    """
    Merge rows with duplicate URLs in `df_completed` by prioritizing data from `df_original`.

    Args:
        df_completed: The processed DataFrame with imputed values.
        df_original: The original DataFrame with raw data.

    Returns:
        A DataFrame with duplicate URLs merged, prioritizing data from the original DataFrame.
    """
    # Identify duplicate URLs
    duplicate_urls = df_completed[df_completed.duplicated(subset=['URL'], keep=False)]
    unique_urls = duplicate_urls['URL'].unique()

    # Initialize a list to store merged rows
    merged_rows = []

    for url in unique_urls:
        # Extract rows with the same URL from both DataFrames
        rows_original = df_original[df_original['URL'] == url]
        rows_completed = df_completed[df_completed['URL'] == url]

        # Start with the row from the original DataFrame if available
        if not rows_original.empty:
            merged_row = rows_original.iloc[0].copy()
        else:
            merged_row = rows_completed.iloc[0].copy()

        # Fill missing values in the merged row with data from the completed DataFrame
        for col in merged_row.index:
            if pd.isna(merged_row[col]) and col in rows_completed.columns:
                merged_row[col] = rows_completed.iloc[0][col]

        # Append the merged row to the list
        merged_rows.append(merged_row)

    # Convert the list of merged rows into a DataFrame
    merged_df = pd.DataFrame(merged_rows)

    # Combine the merged rows with non-duplicate rows from df_completed
    non_duplicate_rows = df_completed[~df_completed['URL'].isin(unique_urls)]
    result_df = pd.concat([non_duplicate_rows, merged_df], ignore_index=True)

    return result_df




In [181]:
merged_df = merge_duplicate_urls(df_completed, derived_url_data)
print(merged_df)

           id                                                URL  URLLength  \
0      226786                          https://www.tetleyusa.com       24.0   
1      163910                 https://www.atlanticwhiteshark.org       33.0   
2      113381  https://apinchek.protationsacc.eu.org/aprep%d0...       59.0   
3      163038                          https://www.malankara.com       24.0   
4      117472                               https://www.rapp.com       19.0   
...       ...                                                ...        ...   
77622  159262                    https://www.physicianonfire.com       31.0   
77623  118490     https://www.golfe-saint-tropez-information.com       45.0   
77624   77235                          https://www.zappallas.com       25.0   
77625  128558                           https://www.swoonish.com       24.0   
77626  122751  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   

                                              Domai

In [182]:
rows_with_duplicate_urls = merged_df[merged_df['URL'].isin(duplicate_urls)]
print(rows_with_duplicate_urls)

           id                                                URL  URLLength  \
77626  122751  http://test-mantenimiento-bancaweb.azurewebsit...       53.0   

                                              Domain  DomainLength  \
77626  test-mantenimiento-bancaweb.azurewebsites.net          45.0   

       IsDomainIP  TLD  CharContinuationRate  TLDLegitimateProb  URLCharProb  \
77626       False  net              0.320755           0.029014     0.061098   

       ...    Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS     NoOfJS  \
77626  ...  False   False              True  41.483831      1.0  16.370174   

       NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
77626          1.0           0.0             38.0  False  

[1 rows x 55 columns]


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [183]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77627 entries, 0 to 77626
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77627 non-null  int64  
 1   URL                         77627 non-null  object 
 2   URLLength                   77627 non-null  float64
 3   Domain                      77627 non-null  object 
 4   DomainLength                77627 non-null  float64
 5   IsDomainIP                  77627 non-null  boolean
 6   TLD                         77627 non-null  object 
 7   CharContinuationRate        77627 non-null  float64
 8   TLDLegitimateProb           77627 non-null  float64
 9   URLCharProb                 77627 non-null  float64
 10  TLDLength                   77627 non-null  float64
 11  NoOfSubDomain               77627 non-null  float64
 12  HasObfuscation              77627 non-null  boolean
 13  NoOfObfuscatedChar          776

In [184]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from scipy.stats import chi2_contingency

def feature_selection(df, target_column, variance_threshold=0.1, correlation_threshold=0.75, max_categories=50):
    """
    Performs robust feature selection with detailed logging.

    Args:
        df (pd.DataFrame): Input dataframe
        target_column (str): Name of the target column
        variance_threshold (float): Minimum variance threshold for numeric features
        correlation_threshold (float): Maximum correlation threshold for numeric features
        max_categories (int): Maximum number of unique categories to process

    Returns:
        dict: Dictionary containing selected features and details of dropped features with reasons
    """
    # Make a copy to avoid modifying original data
    df = df.copy()

    # Separate features and target
    X = df.drop(columns=[target_column])
    y = df[target_column]

    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object', 'category', 'bool', 'boolean']).columns

    selected_features = []
    dropped_features = []

    # Process numeric features
    if len(numeric_features) > 0:
        # Variance threshold
        selector = VarianceThreshold(threshold=variance_threshold)
        selector.fit(X[numeric_features])
        variance_selected = numeric_features[selector.get_support()]

        dropped_due_to_variance = list(set(numeric_features) - set(variance_selected))
        dropped_features.extend([(col, 'Low Variance') for col in dropped_due_to_variance])

        if len(variance_selected) > 0:
            # Correlation with target
            target_corr = X[variance_selected].corrwith(y).abs()
            high_corr_with_target = target_corr[target_corr >= correlation_threshold].index.tolist()

            dropped_due_to_low_target_corr = list(set(variance_selected) - set(high_corr_with_target))
            dropped_features.extend([(col, 'Low Correlation with Target') for col in dropped_due_to_low_target_corr])

            # Correlation analysis among selected features
            X_high_corr = X[high_corr_with_target]
            corr_matrix = X_high_corr.corr()

            drop_features = set()
            for i in range(len(corr_matrix.columns)):
                if corr_matrix.columns[i] not in drop_features:
                    for j in range(i + 1, len(corr_matrix.columns)):
                        if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
                            drop_features.add(corr_matrix.columns[j])

            correlation_selected = [f for f in high_corr_with_target if f not in drop_features]

            dropped_due_to_feature_corr = list(set(high_corr_with_target) - set(correlation_selected))
            dropped_features.extend([(col, 'High Correlation with Other Features') for col in dropped_due_to_feature_corr])

            if len(correlation_selected) > 0:
                # Mutual information
                mi_scores = mutual_info_classif(X[correlation_selected], y, random_state=42)
                mi_selected = [feature for score, feature in 
                               zip(mi_scores, correlation_selected) if score > 0.01]

                dropped_due_to_low_mi = list(set(correlation_selected) - set(mi_selected))
                dropped_features.extend([(col, 'Low Mutual Information') for col in dropped_due_to_low_mi])

                selected_features.extend(mi_selected)

    # Process categorical features
    if len(categorical_features) > 0:
        for feature in categorical_features:
            # Skip if too many unique categories
            if X[feature].nunique() > max_categories:
                dropped_features.append((feature, 'Too Many Unique Categories'))
                continue

            try:
                # Convert boolean to int for contingency table
                if X[feature].dtype in ['bool', 'boolean']:
                    contingency = pd.crosstab(X[feature].astype(int), y)
                else:
                    contingency = pd.crosstab(X[feature], y)

                chi2, p_value, _, _ = chi2_contingency(contingency)

                # Select feature if statistically significant
                if p_value < 0.05:
                    selected_features.append(feature)
                else:
                    dropped_features.append((feature, 'High p-value in Chi-Square Test'))
            except (ValueError, MemoryError):
                dropped_features.append((feature, 'Error in Processing'))
                continue

    return {
        'selected_features': selected_features,
        'dropped_features': dropped_features
    }

def print_feature_importance(df, selected_features, target_column):
    """
    Prints importance metrics for selected features.

    Args:
        df (pd.DataFrame): Input dataframe
        selected_features (list): List of selected feature names
        target_column (str): Name of the target column
    """
    df = df.copy()

    print("\nFeature Importance Summary:")
    print("-" * 50)

    feature_importance = []
    for feature in selected_features:
        if df[feature].dtype in ['int64', 'float64']:
            mi_score = mutual_info_classif(df[[feature]], df[target_column], random_state=42)[0]
            feature_importance.append((feature, mi_score, 'MI'))
            print(f"{feature:30} MI Score: {mi_score:.4f}")
        else:
            if df[feature].dtype in ['bool', 'boolean']:
                contingency = pd.crosstab(df[feature].astype(int), df[target_column])
            else:
                contingency = pd.crosstab(df[feature], df[target_column])
            chi2, p_value, _, _ = chi2_contingency(contingency)
            feature_importance.append((feature, chi2, 'Chi2'))
            print(f"{feature:30} Chi2 Score: {chi2:.4f} (p={p_value:.4e})")

    return sorted(feature_importance, key=lambda x: x[1], reverse=True)


In [189]:
def generate_polynomial_features(df, selected_features, degree=2):
    """
    Generate polynomial features for selected numerical features, with fixes for NaN and non-numeric data.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        selected_features (list): List of selected numerical features
        degree (int): The degree of polynomial features to generate
    
    Returns:
        pd.DataFrame: DataFrame with added polynomial features
    """
    from sklearn.preprocessing import PolynomialFeatures

    # Filter numerical features
    numerical_features = df[selected_features].select_dtypes(include=['int64', 'float64'])

    # Handle missing values by imputing them with the mean
    numerical_features = numerical_features.fillna(numerical_features.mean())

    # Generate polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly.fit_transform(numerical_features)

    # Get the names of polynomial features
    poly_feature_names = poly.get_feature_names_out(numerical_features.columns)

    # Create a DataFrame with new features
    poly_df = pd.DataFrame(poly_features, columns=poly_feature_names, index=df.index)

    # Combine the original DataFrame with the polynomial features
    return pd.concat([df, poly_df], axis=1)




In [190]:
def generate_interaction_features(df, selected_features):
    """
    Generate interaction features between selected numerical features.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        selected_features (list): List of selected numerical features
    
    Returns:
        pd.DataFrame: DataFrame with added interaction features
    """
    interaction_features = {}
    for i, feature1 in enumerate(selected_features):
        for feature2 in selected_features[i+1:]:
            interaction_name = f"{feature1}_x_{feature2}"
            interaction_features[interaction_name] = df[feature1] * df[feature2]
    
    interaction_df = pd.DataFrame(interaction_features)
    return pd.concat([df, interaction_df], axis=1)


In [194]:
# Seleksi fitur
target_column = 'label'
feature_selection_result = feature_selection(merged_df, target_column)
selected_features = feature_selection_result['selected_features']

In [196]:
print(merged_df.isnull().sum())

id                                0
URL                               0
URLLength                         0
Domain                            0
DomainLength                      0
IsDomainIP                        0
TLD                               0
CharContinuationRate              0
TLDLegitimateProb                 0
URLCharProb                       0
TLDLength                         0
NoOfSubDomain                     0
HasObfuscation                    0
NoOfObfuscatedChar                0
ObfuscationRatio                  0
NoOfLettersInURL                  0
LetterRatioInURL                  0
NoOfDegitsInURL                   0
DegitRatioInURL                   0
NoOfEqualsInURL                   0
NoOfQMarkInURL                    0
NoOfAmpersandInURL                0
NoOfOtherSpecialCharsInURL        0
SpacialCharRatioInURL             0
IsHTTPS                           0
LineOfCode                        0
LargestLineLength                 0
HasTitle                    

In [195]:
df_with_poly = generate_polynomial_features(merged_df, selected_features, degree=2)

ValueError: at least one array or dtype is required

In [ ]:
df_with_interactions = generate_interaction_features(df_with_poly, selected_features)

In [ ]:
print(f"Dataset after adding features: {df_with_interactions.shape}")

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [17]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [18]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [19]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [20]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [21]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [22]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [23]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [24]:
# Type your code here

## B. Naive Bayes

In [25]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [26]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [27]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`